In [1]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

os.chdir("/content/drive/My Drive/Colab Notebooks/cs7643")
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/My Drive/Colab Notebooks/cs7643


In [3]:
import os
import urllib.request
import tarfile

# URL of the Hateful Memes features archive
URL = "https://dl.fbaipublicfiles.com/mmf/data/datasets/hateful_memes/defaults/features/features_2020_10_01.tar.gz"
TAR_PATH = "features_2020_10_01.tar.gz"
EXTRACT_DIR = "detectron.lmdb"

# 1. Download the tar.gz file (if not already downloaded)
if not os.path.exists(TAR_PATH):
    print(f"Downloading from {URL} ...")
    urllib.request.urlretrieve(URL, TAR_PATH)
    print(f"Downloaded to {TAR_PATH}")
else:
    print(f"{TAR_PATH} already exists, skipping download.")

# 2. Extract the tar.gz file (if not already extracted)
if not os.path.exists(EXTRACT_DIR):
    print(f"Extracting {TAR_PATH} ...")
    with tarfile.open(TAR_PATH, "r:gz") as tar:
        tar.extractall()
    print(f"Extraction complete. Files are in: {EXTRACT_DIR}/")
else:
    print(f"{EXTRACT_DIR}/ already exists, skipping extraction.")

features_2020_10_01.tar.gz already exists, skipping download.
detectron.lmdb/ already exists, skipping extraction.


In [4]:
!pip install lmdb

In [5]:
import torch
import numpy as np
import random
import os

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)               # for single GPU
    torch.cuda.manual_seed_all(seed)           # if using multi-GPU

    # Ensure deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # For some CUDA operations (e.g., DataLoader workers)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)


In [6]:
import lmdb
import torch
import pickle
from torch.utils.data import Dataset


class HatefulMemesDataset(Dataset):
    def __init__(self, hf_split, lmdb_path, tokenizer):
        """
        hf_split: one split from the HF DatasetDict (e.g. hf_ds['train'])
        lmdb_env: opened lmdb.Environment
        tokenizer: HuggingFace tokenizer (optional)
        """
        self.data = hf_split
        self.lmdb_path = lmdb_path
        self.env = None
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def _get_image_id(self, img_path):
        # "img/40259.png" -> "40259"
        return img_path.split("/")[-1].split(".")[0]

    def _load_visual_feats(self, img_id):
        if self.env is None:  # opened separately in each worker
            self.env = lmdb.open(
                self.lmdb_path,
                readonly=True,
                lock=False,
                readahead=False,
                meminit=False,
            )
        with self.env.begin(write=False) as txn:
            buf = txn.get(img_id.encode("utf-8"))
            sample = pickle.loads(buf)
        feats = torch.tensor(
            sample["features"], dtype=torch.float32
        )  # (num_boxes, 2048)
        bbox = torch.tensor(sample["bbox"], dtype=torch.float32)  # (num_boxes, 4)
        return feats, bbox

    def __getitem__(self, idx):
        row = self.data[idx]
        text = row["text"]
        img_path = row["img"]
        label = row["label"]

        img_id = self._get_image_id(img_path)
        visual_embeds, _ = self._load_visual_feats(img_id)

        encoded = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=48,
            return_tensors="pt",
        )

        item = {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "token_type_ids": encoded["token_type_ids"].squeeze(0),
            "visual_embeds": visual_embeds,
            "visual_attention_mask": torch.ones(
                visual_embeds.size(0), dtype=torch.long
            ),
            "visual_token_type_ids": torch.zeros(
                visual_embeds.size(0), dtype=torch.long
            ),
            "label": torch.tensor(label, dtype=torch.long),
        }

        return item

In [7]:
!pip install datasets

In [8]:
import pandas as pd
from datasets import Dataset, load_dataset
from torch.utils.data import DataLoader
from transformers import VisualBertModel, VisualBertConfig, BertTokenizer

dataset = load_dataset("neuralcatcher/hateful_memes")
# Remove duplicates
for i_split, i_data in dataset.items():
    dataset[i_split] = Dataset.from_pandas(
        pd.DataFrame(i_data).drop_duplicates(), preserve_index=False
    )

lmdb_path = "detectron.lmdb"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

loader_train = DataLoader(
    HatefulMemesDataset(dataset["train"], lmdb_path, tokenizer),
    batch_size=32,
    shuffle=True,
    num_workers=8,
)

loader_validation = DataLoader(
    HatefulMemesDataset(dataset["validation"], lmdb_path, tokenizer),
    batch_size=265,
    shuffle=False,
    num_workers=8,
)

loader_test = DataLoader(
    HatefulMemesDataset(dataset["test"], lmdb_path, tokenizer),
    batch_size=265,
    shuffle=False,
    num_workers=8,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [9]:
import torch.nn as nn
from transformers import VisualBertModel

def freeze_all_but_last_n_layers(model, n):
    # freeze everything
    for p in model.parameters():
        p.requires_grad = False

    # unfreeze last n encoder layers
    total = len(model.encoder.layer)
    for i in range(total - n, total):
        for p in model.encoder.layer[i].parameters():
            p.requires_grad = True

    # unfreeze pooler (optional)
    for p in model.pooler.parameters():
        p.requires_grad = True

class VisualBertForClassification(nn.Module):
    def __init__(self, num_labels=2):
        super().__init__()
        self.visualbert = VisualBertModel.from_pretrained(
            "uclanlp/visualbert-vqa-coco-pre"
        )
        freeze_all_but_last_n_layers(self.visualbert, n=9)
        hidden_size = self.visualbert.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(
        self,
        input_ids,
        attention_mask,
        token_type_ids,
        visual_embeds,
        visual_attention_mask,
        visual_token_type_ids,
    ):
        outputs = self.visualbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            visual_embeds=visual_embeds,
            visual_attention_mask=visual_attention_mask,
            visual_token_type_ids=visual_token_type_ids,
        )

        pooled = outputs.pooler_output  # (B, 768)
        logits = self.classifier(pooled)
        return logits

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

model = VisualBertForClassification().to(device)

optimizer = torch.optim.AdamW([
    {"params": model.visualbert.parameters(), "lr": 2e-5, "weight_decay": 0},
    {"params": model.classifier.parameters(),  "lr": 2e-3, "weight_decay": 1e-5},
])
class_weights = torch.tensor([0.7754, 1.4077], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.00)

Using cuda


In [11]:
best_auc = 0.0
best_model_path = "best_visualbert.pt"

In [12]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

keys2pass = [
    "input_ids",
    "attention_mask",
    "token_type_ids",
    "visual_embeds",
    "visual_attention_mask",
    "visual_token_type_ids",
]
model.eval()
all_preds_validation = []
all_probs_validation = []
all_labels_validation = []
val_loss = 0
with torch.inference_mode():
    for batch in loader_validation:
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**{k: batch[k] for k in keys2pass})
        loss = criterion(logits, batch["label"])
        val_loss += loss.item()

        probs = logits.softmax(dim=1)[:, 1].cpu().numpy()
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = batch["label"].cpu().numpy()

        all_probs_validation.extend(probs)
        all_preds_validation.extend(preds)
        all_labels_validation.extend(labels)

acc = accuracy_score(all_labels_validation, all_preds_validation)
f1 = f1_score(all_labels_validation, all_preds_validation)
auc = roc_auc_score(all_labels_validation, all_probs_validation)
print("Epoch 0")
print(f"Val Loss: {val_loss:.4f} | ACC: {acc:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")
for epoch in range(20):
    model.train()
    all_preds_train = []
    all_probs_train = []
    all_labels_train = []
    total_loss = 0
    print(f"Epoch {epoch+1}")
    for batch in loader_train:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        logits = model(**{k: batch[k] for k in keys2pass})
        loss = criterion(logits, batch["label"])
        loss.backward()
        # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()

        probs = logits.softmax(dim=1)[:, 1].detach().cpu().numpy()
        preds = logits.argmax(dim=1).detach().cpu().numpy()
        labels = batch["label"].cpu().numpy()

        all_probs_train.extend(probs)
        all_preds_train.extend(preds)
        all_labels_train.extend(labels)
    acc = accuracy_score(all_labels_train, all_preds_train)
    f1 = f1_score(all_labels_train, all_preds_train)
    auc = roc_auc_score(all_labels_train, all_probs_train)
    print(f"Train Loss: {total_loss:.4f} | ACC: {acc:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")
    # ---- VALIDATION ----
    model.eval()
    all_preds_validation = []
    all_probs_validation = []
    all_labels_validation = []
    val_loss = 0
    with torch.inference_mode():
        for batch in loader_validation:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**{k: batch[k] for k in keys2pass})
            loss = criterion(logits, batch["label"])
            val_loss += loss.item()

            probs = logits.softmax(dim=1)[:, 1].cpu().numpy()
            preds = logits.argmax(dim=1).cpu().numpy()
            labels = batch["label"].cpu().numpy()

            all_probs_validation.extend(probs)
            all_preds_validation.extend(preds)
            all_labels_validation.extend(labels)

    acc = accuracy_score(all_labels_validation, all_preds_validation)
    f1 = f1_score(all_labels_validation, all_preds_validation)
    auc = roc_auc_score(all_labels_validation, all_probs_validation)
    print(f"Val Loss: {val_loss:.4f} | ACC: {acc:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")

    # Save best model based on AUC
    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), best_model_path)
print(f"--> Saved best model (AUC={best_auc:.4f})")


Epoch 0
Val Loss: 2.0465 | ACC: 0.6141 | F1: 0.0000 | AUC: 0.5029
Epoch 1
Train Loss: 169.5963 | ACC: 0.6400 | F1: 0.5358 | AUC: 0.6860
Val Loss: 2.0035 | ACC: 0.6109 | F1: 0.3882 | AUC: 0.6360
Epoch 2
Train Loss: 137.9868 | ACC: 0.7588 | F1: 0.6718 | AUC: 0.8182
Val Loss: 2.0039 | ACC: 0.6297 | F1: 0.4925 | AUC: 0.6661
Epoch 3
Train Loss: 113.7144 | ACC: 0.8075 | F1: 0.7408 | AUC: 0.8842
Val Loss: 2.0152 | ACC: 0.6531 | F1: 0.5316 | AUC: 0.6868
Epoch 4
Train Loss: 89.0332 | ACC: 0.8562 | F1: 0.8089 | AUC: 0.9314
Val Loss: 2.6209 | ACC: 0.6641 | F1: 0.5316 | AUC: 0.6831
Epoch 5
Train Loss: 70.1693 | ACC: 0.8845 | F1: 0.8462 | AUC: 0.9571
Val Loss: 2.9701 | ACC: 0.6656 | F1: 0.4541 | AUC: 0.6900
Epoch 6
Train Loss: 55.6254 | ACC: 0.9118 | F1: 0.8818 | AUC: 0.9730
Val Loss: 2.5857 | ACC: 0.6766 | F1: 0.5470 | AUC: 0.6884
Epoch 7
Train Loss: 42.5944 | ACC: 0.9333 | F1: 0.9098 | AUC: 0.9837
Val Loss: 3.0347 | ACC: 0.6609 | F1: 0.4642 | AUC: 0.6903
Epoch 8
Train Loss: 35.2333 | ACC: 0.9445 

In [13]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

model = VisualBertForClassification().to(device)
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()
all_preds = []
all_probs = []
all_labels = []
test_loss = 0

with torch.inference_mode():
    for batch in loader_test:
        batch = {k: v.to(device) for k, v in batch.items()}

        logits = model(**{k: batch[k] for k in keys2pass})
        loss = criterion(logits, batch["label"])
        test_loss += loss.item()

        probs = logits.softmax(dim=1)[:, 1].cpu().numpy()
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = batch["label"].cpu().numpy()

        all_probs.extend(probs)
        all_preds.extend(preds)
        all_labels.extend(labels)

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
auc = roc_auc_score(all_labels, all_probs)

print("Test Loss:", test_loss)
print("Test Acc:", acc)
print("Test F1:", f1)
print("Test AUC:", auc)

Test Loss: 26.971502661705017
Test Acc: 0.682
Test F1: 0.5364431486880467
Test AUC: 0.719469620601173
